## NoteBook

In [ ]:
!pip install "sagemaker>=2.48.0" --upgrade

In [ ]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# Huggin Face Base 모델 로드
classifier1_model = HuggingFaceModel(
   model_data="s3://{버킷 이름}/classifier1.tar.gz",  # 1번 의도 분류 모델 전체 채팅 -> (일반, (질문 & 요청))
   role=role, # Endpoint 생성 권한이 있는 role
   transformers_version="4.26",
   pytorch_version="1.13",
   py_version="py39",
)

In [ ]:
# SageMaker 에 모델 배포
classifier1 = classifier1_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

In [ ]:
# Huggin Face Base 모델 로드
classifier2_model = HuggingFaceModel(
   model_data="s3://{버킷 이름}/classifier2.tar.gz",  # 2번 의도 분류 모델 (질문 & 요청) 채팅 -> 질문, 요청
   role=role,
   transformers_version="4.26",
   pytorch_version="1.13",
   py_version="py39",
)

In [ ]:
# SageMaker 에 모델 배포
classifier2 = classifier2_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

## Lambda

In [ ]:
import json
import boto3

runtime = boto3.client("runtime.sagemaker")
# SageMaker Endpoint 이름
classifier_1_endpoint_name  = "huggingface-pytorch-inference-2023-08-29-01-43-00-324"
classifier_2_endpoint_name = "huggingface-pytorch-inference-2023-08-29-01-45-33-676"

# API 요청 처리
def lambda_handler(event, context):
    chat_list = event["list"]
    
    # 채팅 리스트
    chat_message_list = [chat["message"] for chat in chat_list]
    
    # 1번 분류 전체 채팅 -> (일반, (질문 & 요청))
    classifier_1_payload = {"inputs" : chat_message_list}
    
    classifier_1_response = runtime.invoke_endpoint(
                                    EndpointName=classifier_1_endpoint_name, 
                                    ContentType="application/json", 
                                    Body=json.dumps(classifier_1_payload)
                                )
    
    classifier_1_result_list = json.loads(classifier_1_response["Body"].read().decode())
    
    # 결과 처리
    result = dict()
    result["chat_data"] = list()
    
    for chat_idx in range(len(chat_list)):
        
        chat = chat_list[chat_idx]
        
        classifier_result = dict()
        classifier_result["commentNo"] = chat["commentNo"]
        classifier_result["nickname"] = chat["nickname"]
        classifier_result["message"] = chat["message"]
        
        if classifier_1_result_list[chat_idx]["label"] == "LABEL_0":
            classifier_result["result"] = "일반"
        else:
            # 2번 분류 (질문 & 요청) 채팅 -> 질문, 요청
            classifier_2_payload = {"inputs" : chat["message"]}
            
            classifier_2_response = runtime.invoke_endpoint(
                                    EndpointName=classifier_2_endpoint_name, 
                                    ContentType="application/json", 
                                    Body=json.dumps(classifier_2_payload)
                                )
            
            classifier_2_result = json.loads(classifier_2_response["Body"].read().decode())[0]
            
            if classifier_2_result["label"] == "LABEL_0":
                classifier_result["result"] = "질문"
            else:
                classifier_result["result"] = "요청"
    
        
        result["chat_data"].append(classifier_result)
    
    return result